In [16]:
import pandas as pd
import tensorflow as tf
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_Internal_dataset_editgbm.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#X 표준화

scaler = MinMaxScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=10)

start_time = time.time()

#모델 생성과 시행

gb_clf = GradientBoostingClassifier(random_state=10)
gb_clf.fit(X_train, Y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(Y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행시간: {0:.1f}초'.format(time.time() - start_time))

#파라미터 튜닝

param = {}
grid_clf = GridSearchCV(gb_clf, param_grid=param, cv=5, verbose=1)
grid_clf.fit(X_train, Y_train)
print('최적 하이퍼 파라미터:\n',grid_clf.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_clf.best_score_))

#GBM 정확도: 0.8571
#GBM 수행시간: 4.2초

GBM 정확도: 1.0000
GBM 수행시간: 0.2초
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.9s finished


최적 하이퍼 파라미터:
 {}
최고 예측 정확도: 0.7000


In [2]:
import pandas as pd
import tensorflow as tf
import time
import numpy as np
import seaborn as sns
import pickle
import joblib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, cross_val_predict, KFold
from sklearn.metrics import accuracy_score, confusion_matrix ,multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler


#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_Internal_dataset_editgbm.csv')
df2 = pd.read_csv('./VRK2_inhibitor_descriptor_external_dataset_editgbm.csv')
df3 = pd.concat([df, df2]) #total data는 internal과 external concat하면 혼동함수 만들 수 있다.

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

dataset2 = df2.values
exX = dataset2[:,0:-1]
exY = dataset2[:,-1]

dataset3 = df3.values
totX = dataset3[:,0:-1]
totY = dataset3[:,-1]

#X 표준화

scaler = MinMaxScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

exX_scaled = scaler.transform(exX)

totX_scaled = scaler.transform(totX)

#학습셋과 테스트셋을 나누고  crossval 실행
#skf = KFold(n_splits=5, random_state=10)
#n_iter = 0
#cv_accuracy = []
#cm_holder = []
#gb_clf = GradientBoostingClassifier(random_state=10)

#for train_index, test_index in skf.split(X_scaled, Y):
#    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
#    Y_train, Y_test = Y[train_index], Y[test_index]
#    gb_clf.fit(X_train, Y_train)
#    pred = gb_clf.predict(X_test)
    
#    n_iter += 1
#    accuracy = np.round(accuracy_score(Y_test, pred), 5)
#    train_size = X_train.shape[0]
#    test_size = X_test.shape[0]
#    print('\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'.format(n_iter, accuracy, train_size, test_size))
#    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
#    print('#{0} 검증 세트 : {1}'.format(n_iter, Y_test))
#    print('#{0} 검증 세트 예측 결과 : {1}'.format(n_iter, pred))
#    cv_accuracy.append(accuracy)
#    cm_holder.append(confusion_matrix(Y_test, pred, labels=[0,1,2,3]))

#print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 5))
#print('## 평균 검증 정확도:', np.mean(cv_accuracy))
#print(cm_holder)

#검증이 끝난 모델을 저장하고 로드해서 conf matrix를 구한다.

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

gb_clf = GradientBoostingClassifier(random_state=10)
gb_clf.fit(X_train, Y_train)
#start_time = time.time()

#print('GBM 내부 정확도: {0:.4f}'.format(gb_accuracy))
#print('GBM 수행시간: {0:.1f}초'.format(time.time() - start_time))

#confision matrix   

#conf_matrix1_1 = confusion_matrix(Y_test, gb_pred1,labels=[0,1,2,3])
#print(conf_matrix1_1)
#conf_matrix1_2 = confusion_matrix(Y_test2, gb_pred2,labels=[0,1,2,3])
#print(conf_matrix1_2)
#conf_matrix1_3 = confusion_matrix(Y_test3, gb_pred3,labels=[0,1,2,3])
#print(conf_matrix1_3)
#conf_matrix1_4 = confusion_matrix(Y_test4, gb_pred4,labels=[0,1,2,3])
#print(conf_matrix1_4)
#conf_matrix1_5 = confusion_matrix(Y_train4, gb_pred5,labels=[0,1,2,3])
#print(conf_matrix1_5)
#conf_matrix1 = conf_matrix1_1+ conf_matrix1_2 + conf_matrix1_3 + conf_matrix1_4 + conf_matrix1_5
#print(conf_matrix1)

pred = gb_clf.predict(X)
accuracy = accuracy_score(Y, pred)
print('GBM 내부 정확도: {0:.4f}'.format(accuracy))
conf_matrix = confusion_matrix(Y, pred,labels=[0,1,2,3])
print(conf_matrix)
conf_matrix_nor = confusion_matrix(Y, pred,labels=[0,1,2,3])/26
print(conf_matrix_nor)

pred2 = gb_clf.predict(exX)
accuracy2 = accuracy_score(exY, pred2)
print('GBM 외부 정확도: {0:.4f}'.format(accuracy2))
conf_matrix2 = confusion_matrix(exY, pred2,labels=[0,1,2,3])
print(conf_matrix2)
conf_matrix2_nor = confusion_matrix(exY, pred2,labels=[0,1,2,3])/6
print(conf_matrix2_nor)

pred3 = gb_clf.predict(totX)
accuracy3 = accuracy_score(totY, pred3)
print('GBM 전체 정확도: {0:.4f}'.format(accuracy3))
conf_matrix3 = confusion_matrix(totY, pred3,labels=[0,1,2,3])
print(conf_matrix3)
conf_matrix3_nor = confusion_matrix(totY, pred3,labels=[0,1,2,3])/32
print(conf_matrix3_nor)

#fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize=(15, 20))
#sns.heatmap(conf_matrix, annot=True, ax=ax[0][0])
#ax[0][0].set_title('Internal confusion matrix', pad = 12)
#sns.heatmap(conf_matrix_nor, annot=True, ax=ax[1][0])
#ax[1][0].set_title('Internal confusion matrix(Normalized)', pad = 12)
#sns.heatmap(conf_matrix2, annot=True, ax=ax[0][1])
#ax[0][1].set_title('External confusion matrix', pad = 12)
#sns.heatmap(conf_matrix2_nor, annot=True, ax=ax[1][1])
#ax[1][1].set_title('External confusion matrix(Normalized)', pad = 12)
#sns.heatmap(conf_matrix3, annot=True, ax=ax[2][0])
#ax[2][0].set_title('Total confusion matrix', pad = 12)
#sns.heatmap(conf_matrix3_nor, annot=True, ax=ax[2][1])
#ax[2][1].set_title('Total confusion matrix(Normalized)', pad = 12)

#모델 저장
#saved_model_rf = joblib.dump(gb_clf,'./VRK2_GBM.pkl')

GBM 내부 정확도: 1.0000
[[ 6  0  0  0]
 [ 0  2  0  0]
 [ 0  0  8  0]
 [ 0  0  0 10]]
[[0.23076923 0.         0.         0.        ]
 [0.         0.07692308 0.         0.        ]
 [0.         0.         0.30769231 0.        ]
 [0.         0.         0.         0.38461538]]
GBM 외부 정확도: 0.3333
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 2]
 [0 0 0 2]]
[[0.         0.         0.         0.16666667]
 [0.         0.         0.         0.16666667]
 [0.         0.         0.         0.33333333]
 [0.         0.         0.         0.33333333]]
GBM 전체 정확도: 0.8750
[[ 6  0  0  1]
 [ 0  2  0  1]
 [ 0  0  8  2]
 [ 0  0  0 12]]
[[0.1875  0.      0.      0.03125]
 [0.      0.0625  0.      0.03125]
 [0.      0.      0.25    0.0625 ]
 [0.      0.      0.      0.375  ]]


In [138]:
import pandas as pd
import tensorflow as tf
import time
import warnings
import matplotlib.pyplot as plt
import sys
warnings.filterwarnings('ignore')

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_Internal_dataset_editgbm.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#X 정규화

scaler = MinMaxScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=4)

start_time = time.time()

#모델 생성과 시행

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, Y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(Y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행시간: {0:.1f}초'.format(time.time() - start_time))


#importance = gb_clf.feature_importances_

# summarize feature importance

#sys.stdout = open('class4_nonzero.txt', 'w') 

#for i,v in enumerate(importance):
#    print('%0d,%.5f' % (i,v))
    
#sys.stdout.close()      
       
# plot feature importance
#plt.bar([x for x in range(len(importance))], importance)
#plt.axis([1725, 1800, 0, 0.01])
#plt.show()

GBM 정확도: 0.8333
GBM 수행시간: 0.4초


In [21]:
import numpy as np

np.arange(0.1, 0.5, 0.05)

#list(range(1,5))

array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45])

In [3]:
import pandas as pd
import tensorflow as tf
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#데이터 입력
df = pd.read_csv('./VRK2_inhibitor_descriptor_Internal_dataset_editgbm.csv')

#데이터 분류
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#학습셋과 테스트셋을 나눔

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

start_time = time.time()

#모델 생성과 시행

gb_clf = GradientBoostingClassifier(learning_rate=0.19, max_depth=1, min_samples_leaf=3, min_samples_split=2, 
                                    n_estimators=4, subsample=0.5, random_state=10)
gb_clf.fit(X_train, Y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(Y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행시간: {0:.1f}초'.format(time.time() - start_time))

GBM 정확도: 0.6667
GBM 수행시간: 0.0초


In [13]:
import sklearn
sklearn.__version__

'0.23.2'